In [1]:
import xarray as xr

In [3]:
data = xr.open_dataset("data/2011_t2m_era5_2deg.nc")
data

<xarray.Dataset> Size: 95MB
Dimensions:     (valid_time: 1460, lon: 180, lat: 90)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 12kB 2011-01-01 ... 2011-12-31T18...
  * lon         (lon) float64 1kB 0.0 2.0 4.0 6.0 ... 352.0 354.0 356.0 358.0
  * lat         (lat) float64 720B -89.0 -87.0 -85.0 -83.0 ... 85.0 87.0 89.0
Data variables:
    t2m         (valid_time, lat, lon) float32 95MB ...
Attributes:
    CDI:                     Climate Data Interface version 2.4.4 (https://mp...
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    history:                 Wed Jun 11 23:24:35 2025: cdo remapbil,r180x90 2...
    CDO:                     Climate Data Operators version 2.4.4 (https://mp...

In [ ]:
def extract_month(data):
    return 

In [11]:
data.valid_time.dt.month.values

array([ 1,  1,  1, ..., 12, 12, 12], shape=(1460,))

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

weather_data = xr.open_dataset("data/2011_t2m_era5_2deg.nc").t2m.isel(valid_time=slice(0, 2)).values
# convert to tensor
weather_data = torch.tensor(weather_data)

B, l, L = weather_data.shape
# flatten to convert to tokens
weather_data = weather_data.reshape(B, l*L)

class BasicSelfAttention(nn.Module):
    def __init__(self, embed_dim):
        super().__init__()
        self.qkv = nn.Linear(embed_dim, 3*embed_dim)
        self.out_proj = nn.Linear(embed_dim, embed_dim)
    def forward(self, x):
        qkv = self.qkv(x)
        q, k, v = qkv.split(3, dim=-1)
        attn_weights = q @ k.transpose(-2, -1)
        attn_weights = attn_weights / math.sqrt(q.shape[-1])
        attn_weights = F.softmax(attn_weights, dim=-1)
        attn_weights = self.out_proj(attn_weights)
        return attn_weights
        


In [20]:
B, l, L

(2, 90, 180)

In [15]:
weather_data.t2m.isel(valid_time=0).values

array([[246.6997 , 246.69385, 246.68799, ..., 246.74463, 246.729  ,
        246.71533],
       [247.52002, 247.4751 , 247.43018, ..., 247.52197, 247.49854,
        247.51025],
       [250.76611, 250.26807, 249.72314, ..., 251.37158, 251.1919 ,
        250.99072],
       ...,
       [244.43213, 244.16455, 243.84229, ..., 244.36377, 244.43018,
        244.44775],
       [243.59619, 243.6001 , 243.604  , ..., 243.03955, 243.16455,
        243.38135],
       [242.73291, 242.70557, 242.67627, ..., 242.52002, 242.59424,
        242.66455]], shape=(90, 180), dtype=float32)